In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qualang_tools.loops import from_array
import QM
import progress
import time
import h5py

# Analyse des histogrammes de mesure

In [ ]:
# Parameters Definition
n_avg = 20000  # The number of averages
thermalization_time = 200 #in µs
waiting_time = 1 * u.us
###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    I0 = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q0 = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I0_st = declare_stream()  # Stream for the 'I' quadrature
    Q0_st = declare_stream()  # Stream for the 'Q' quadrature
    I1 = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q1 = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I1_st = declare_stream()  # Stream for the 'I' quadrature
    Q1_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
            # Measure the state of the resonator
            # The integration weights have changed to maximize the SNR after having calibrated the IQ blobs.
            measure(
                "readout"*amp(1.),
                "resonator",
                dual_demod.full("cos", "sin", I0),
                dual_demod.full("minus_sin", "cos", Q0),
            )
            align("qubit", "resonator")
            wait(600*u.ns)
            play("x180"*amp(0.72), "qubit" )
            wait(24*u.ns)
            align("qubit", "resonator")
            # The integration weights have changed to maximize the SNR after having calibrated the IQ blobs.
            measure(
                "readout"*amp(1.),
                "resonator",
                dual_demod.full("cos", "sin", I1),
                dual_demod.full("minus_sin", "cos", Q1),
            )
            # Wait for the qubit to decay to the ground state
            wait(thermalization_time * u.us, "resonator")
            # Save the 'I' & 'Q' quadratures to their respective streams
            save(I0, I0_st)
            save(Q0, Q0_st)
            save(I1, I1_st)
            save(Q1, Q1_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I0_st.save_all("I0")
        Q0_st.save_all("Q0")
        I1_st.save_all("I1")
        Q1_st.save_all("Q1")

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

In [ ]:
# Fetch results
I0,Q0,I1,Q1=job.get_results("I0", "Q0","I1","Q1")

In [ ]:
def rot(I,Q,theta,scale=1e4):
    theta = theta*np.pi/180
    return scale*(I*np.cos(theta)-Q*np.sin(theta)) , scale*(I*np.sin(theta)+Q*np.cos(theta))

In [ ]:
fig,ax = plt.subplots()
I0r, Q0r = rot(I0,Q0,0)
I1r, Q1r = rot(I1,Q1,0)
ax.plot(I0r, Q0r, ',', label=r"$|0\rangle$")
ax.plot(I1r, Q1r, ',', label=r"$|1\rangle$")
ax.set_ylabel("Q")
ax.set_xlabel("I")
legend = ax.legend()
for h in legend.legend_handles:
    h.set_marker(".") 
ax.set_aspect("equal")

- Trouvez la rotation pour obtenir toute l'information sur l'état du qubit sur $I$.
- Estimez l'erreur de lecture.
- Quelle est la température du qubit ?

# Trajectoires quantiques

In [ ]:
# Parameters Definition
n_meas = 5000  
###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  
    I = declare(fixed)  
    Q = declare(fixed)  
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_meas, n + 1):  # QUA for_ loop for averaging
        measure(
            "readout",
            "resonator",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        save(I, I_st)
        save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.save_all("I")
        Q_st.save_all("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

In [ ]:
# Fetch results
I,Q=job.get_results("I", "Q")

- A partir de $I$ et $Q$, construisez un vecteur de 0 et de 1 correspondant à l'état du qubit au cours du temps.
- Quelle est la probabilité d’obtenir deux fois de suite des résultats identiques ?
- Quelle est la probabilité d’obtenir deux fois de suite des résultats différents ?
- Quelle est la probabilité de saut de l’état $|0\rangle$ vers l’état $|1\rangle$ ?
- Quelle est la probabilité de saut de l’état $|1\rangle$ vers l’état $|0\rangle$ ?
- Relierz ces deux probabilités à la température du qubit.

# Préparation par la mesure

In [ ]:
# Parameters Definition
n_avg = 500  # The number of averages
# Pulse duration sweep (in clock cycles = 4ns) - must be larger than 4 clock cycles
t_min = 4 // 4
t_max = 1000 // 4
dt = 8 // 4
durations = np.arange(t_min, t_max, dt)

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    t = declare(int)  # QUA variable for the qubit pulse duration
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    Z = declare(int) 
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    update_frequency('resonator',59980000)
    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
        with for_(*from_array(t, durations)):  # QUA for_ loop for sweeping the pulse duration
            # Play the qubit pulse with a variable duration (in clock cycles = 4ns)
            play("pi"*amp(1.0), "qubit", duration=t)
            # Align the two elements to measure after playing the qubit pulse.
            align("qubit", "resonator")
            # Measure the state of the resonator
            measure(
                "readout",
                "resonator",
                dual_demod.full("cos", "sin", I),
                dual_demod.full("minus_sin", "cos", Q),
            )
            # Send back qubit to ground state
            assign(Z, Util.cond(Q>2e-4, -1, 1))
            with if_(Z==1):
                play("pi", "qubit", duration=100*u.ns)
                
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.buffer(len(durations)).average().save("I")
        Q_st.buffer(len(durations)).average().save("Q")      

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

In [ ]:
# Fetch results
I,Q=job.get_results("I", "Q")

In [ ]:
# Create plot
I,Q=job.get_results("I", "Q")
fig,ax = plt.subplots()
ax.plot(4*durations, I*1e4, 4*durations, Q*1e4)
ax.set_ylabel("Quadrature")
ax.set_xlabel("Pulse Duration [ns]")
ax.legend(('I','Q'))

- Cette séquence permet de mesurer les oscillations de Rabi. Pourquoi est-elle beaucoup plus rapide que la première séquence ?
- Vérifiez que les paramètres de préparation par la mesure sont optimisés.
- Modifiez la séquence pour que la QM renvoie le résultat de la mesure de $Z$ au lieu de $I$ et $Q$.
- Utilisez cette séquence plus rapide pour observer les "Rabi chevrons" en variant dans une double boucle la durée et la fréquence de l'impulsion.

# Mesure faible et mesure forte

In [ ]:
# Parameters Definition
n_avg = 2000  # The number of averages
# Pulse duration sweep (in clock cycles = 4ns) - must be larger than 4 clock cycles
phase_2_pi = np.linspace(0, 2, 101)

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    phi = declare(fixed)  # QUA variable for the qubit pulse duration
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    Z = declare(int) 
    Z_st = declare_stream()  # Stream for the readout

    update_frequency('resonator',59980000)
    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
        with for_(*from_array(phi, phase_2_pi)):  
            play("x90"*amp(0.88), "qubit")
            align("qubit", "resonator")
            play("readout"*amp(0.0),"resonator",duration=1000*u.ns)
            align("qubit", "resonator")
            frame_rotation_2pi(phi,"qubit")
            play("x90"*amp(0.88), "qubit")
            align("qubit", "resonator")
            # Measure the state of the resonator
            measure(
                "readout",
                "resonator",
                dual_demod.full("cos", "sin", I),
                dual_demod.full("minus_sin", "cos", Q),
            )
            # Send back qubit to ground state
            assign(Z, Util.cond(Q>2e-4, -1, 1))
            with if_(Z==1):
                play("pi", "qubit", duration=100*u.ns)
                
            save(Z, Z_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        Z_st.buffer(len(phase_2_pi)).average().save("Z")

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

In [ ]:
# Get znd plot results
Z, =job.get_results("Z")
fig,ax = plt.subplots()
ax.plot(phase_2_pi, Z)
ax.set_ylabel(r"$\langle Z \rangle$")
ax.set_xlabel(r"Phase/$2\pi$")

- Modifiez l'amplitude de l'impulsion anvoyée sur le résonateur (ligne ``play("readout"*amp(0.0),"resonator",duration=1000*u.ns)``). Qu'observez-vous ?
- Tracez la cohérence du qubit en fonction de l'amplitude de l'impulsion sur le résonateur.
- Expliquez les termes de mesure "faible" et "forte"

# Tomographie quantique

In [ ]:
# Parameters Definition
n_meas = 5000  
thermalization_time = 200 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    Z = declare(int) 
    Z_st = declare_stream()  # Stream for the readout


    with for_(n, 0, n < n_meas, n + 1):  # QUA for_ loop for averaging
        play("unknown","qubit") 
        # Do something
        measure(
            "readout",
            "resonator",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        # Send back qubit to ground state
        assign(Z, Util.cond(Q>2e-4, -1, 1))
        with if_(Z==1):
            play("pi", "qubit", duration=100*u.ns)
            
        save(Z, Z_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        Z_st.buffer(len(phase_2_pi)).average().save("Z")

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

- L'impulsion `unknown` prépare le qubit dans un état inconnu. Utilisez trois séquences différentes pour mesurer $\langle X \rangle$, $\langle Y \rangle$ et $\langle Z \rangle$ sur l'état inconnu.
- Déduisez-en la matrice densité du qubit dans l'état inconnu.